In [1]:
# Confirm the Spark Connector JAR is available
!ls -al /spark-connector/connector/target/scala-2.12/

total 113432
drwxrwxrwx 1 root root      4096 Oct  4 16:44 .
drwxrwxrwx 1 root root      4096 Sep 26 17:10 ..
drwxrwxrwx 1 root root      4096 Sep 26 17:09 classes
-rwxrwxrwx 1 root root 116153329 Sep 26 17:12 spark-vertica-connector-assembly-3.3.3.jar
drwxrwxrwx 1 root root      4096 Sep 26 17:09 src_managed
drwxrwxrwx 1 root root      4096 Sep 26 17:10 test-classes
drwxrwxrwx 1 root root      4096 Sep 26 17:10 test-zinc
drwxrwxrwx 1 root root      4096 Sep 26 17:09 update
drwxrwxrwx 1 root root      4096 Sep 26 17:09 zinc


In [2]:
# Create the Spark session and context
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .config("spark.master", "spark://spark:7077")
    .config("spark.driver.memory", "2G")
    .config("spark.executor.memory", "1G")
    .config("spark.jars", "/spark-connector/connector/target/scala-2.12/spark-vertica-connector-assembly-3.3.3.jar")
    .getOrCreate())
sc = spark.sparkContext

In [3]:
# Display the context information
print(sc.version)
print(sc.master)
display(sc.getConf().getAll())

3.3.0
spark://spark:7077


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.submitTime', '1665089548075'),
 ('spark.driver.memory', '2G'),
 ('spark.app.initial.jar.urls',
  'spark://e3a78c6272d7:38271/jars/spark-vertica-connector-assembly-3.3.3.jar'),
 ('spark.repl.local.jars',
  'file:///spark-

In [4]:
# Perform a simple write then read using the Spark Connector
columns = ["language", "rating"]
data = [("Scala", 71), ("Java", 89), ("C++", 67), ("Python", 94)]
rdd = sc.parallelize(data)
df = rdd.toDF(columns)

df.write.mode("overwrite").save(format="com.vertica.spark.datasource.VerticaSource",
    host="vertica",
    user="dbadmin",
    password="",
    db="docker",
    table="jupytertest",
    staging_fs_url="webhdfs://hdfs:50070/jupytertest")

df = spark.read.load(format="com.vertica.spark.datasource.VerticaSource",
    host="vertica",
    user="dbadmin",
    password="",
    db="docker",
    table="jupytertest",
    staging_fs_url="webhdfs://hdfs:50070/jupytertest")
df.rdd.collect()
df.show()

+--------+------+
|language|rating|
+--------+------+
|     C++|    67|
|    Java|    89|
|  Python|    94|
|   Scala|    71|
+--------+------+

